In [2]:

import sqlite3
import pandas as pd


In [3]:
# Connect to SQLite database (it will create one if it doesn't exist)
connection = sqlite3.connect('ecommerce_behavior_analysis.db')
cursor = connection.cursor()


In [4]:
# Create Customers table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Customers (
    customer_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    email TEXT,
    city TEXT,
    country TEXT,
    segment TEXT
);
''')

# Create Products table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT NOT NULL,
    category TEXT,
    price DECIMAL(10, 2)
);
''')

# Create Orders table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    order_date TEXT,
    order_value DECIMAL(10, 2),
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);
''')

# Create Order_Details table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Order_Details (
    order_detail_id INTEGER PRIMARY KEY,
    order_id INTEGER,
    product_id INTEGER,
    quantity INTEGER,
    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
    FOREIGN KEY (product_id) REFERENCES Products(product_id)
);
''')

# Insert sample data
# Customers
cursor.execute("INSERT OR IGNORE INTO Customers VALUES (1, 'Alice Smith', 'alice@example.com', 'New York', 'USA', 'Premium');")
cursor.execute("INSERT OR IGNORE INTO Customers VALUES (2, 'Bob Johnson', 'bob@example.com', 'Los Angeles', 'USA', 'Regular');")
cursor.execute("INSERT OR IGNORE INTO Customers VALUES (3, 'Charlie Davis', 'charlie@example.com', 'London', 'UK', 'Premium');")
cursor.execute("INSERT OR IGNORE INTO Customers VALUES (4, 'Diana White', 'diana@example.com', 'Toronto', 'Canada', 'Regular');")

# Products
cursor.execute("INSERT OR IGNORE INTO Products VALUES (1, 'Laptop', 'Electronics', 999.99);")
cursor.execute("INSERT OR IGNORE INTO Products VALUES (2, 'Smartphone', 'Electronics', 699.99);")
cursor.execute("INSERT OR IGNORE INTO Products VALUES (3, 'Headphones', 'Accessories', 199.99);")
cursor.execute("INSERT OR IGNORE INTO Products VALUES (4, 'Coffee Maker', 'Home Appliances', 49.99);")

# Orders
cursor.execute("INSERT OR IGNORE INTO Orders VALUES (1, 1, '2024-01-15', 1199.98);")
cursor.execute("INSERT OR IGNORE INTO Orders VALUES (2, 2, '2024-02-03', 699.99);")
cursor.execute("INSERT OR IGNORE INTO Orders VALUES (3, 3, '2024-03-20', 399.98);")
cursor.execute("INSERT OR IGNORE INTO Orders VALUES (4, 1, '2024-04-05', 49.99);")

# Order_Details
cursor.execute("INSERT OR IGNORE INTO Order_Details VALUES (1, 1, 1, 1);")  # Alice bought 1 Laptop
cursor.execute("INSERT OR IGNORE INTO Order_Details VALUES (2, 1, 3, 1);")  # Alice bought 1 Headphones
cursor.execute("INSERT OR IGNORE INTO Order_Details VALUES (3, 2, 2, 1);")  # Bob bought 1 Smartphone
cursor.execute("INSERT OR IGNORE INTO Order_Details VALUES (4, 3, 3, 2);")  # Charlie bought 2 Headphones
cursor.execute("INSERT OR IGNORE INTO Order_Details VALUES (5, 4, 4, 1);")  # Alice bought 1 Coffee Maker

# Commit the changes
connection.commit()


In [5]:
query = '''
SELECT Customers.name, SUM(Orders.order_value) AS lifetime_value
FROM Customers
JOIN Orders ON Customers.customer_id = Orders.customer_id
GROUP BY Customers.name
ORDER BY lifetime_value DESC;
'''

result = pd.read_sql_query(query, connection)
result


,name,lifetime_value
0,Alice Smith,1249.97
1,Bob Johnson,699.99
2,Charlie Davis,399.98


In [6]:
query = '''
SELECT Customers.segment, AVG(Orders.order_value) AS average_order_value
FROM Customers
JOIN Orders ON Customers.customer_id = Orders.customer_id
GROUP BY Customers.segment;
'''

result = pd.read_sql_query(query, connection)
result


,segment,average_order_value
0,Premium,549.983333
1,Regular,699.990000


In [7]:
query = '''
SELECT Products.product_name, SUM(Order_Details.quantity) AS total_quantity
FROM Products
JOIN Order_Details ON Products.product_id = Order_Details.product_id
GROUP BY Products.product_name
ORDER BY total_quantity DESC
LIMIT 1;
'''

result = pd.read_sql_query(query, connection)
result


,product_name,total_quantity
0,Headphones,3


In [8]:
query = '''
SELECT strftime('%Y-%m', Orders.order_date) AS month, SUM(Orders.order_value) AS total_sales
FROM Orders
GROUP BY month
ORDER BY month;
'''

result = pd.read_sql_query(query, connection)
result


,month,total_sales
0,2024-01,1199.98
1,2024-02,699.99
2,2024-03,399.98
3,2024-04,49.99


In [9]:
query = '''
SELECT Customers.name, COUNT(Orders.order_id) AS total_orders
FROM Customers
JOIN Orders ON Customers.customer_id = Orders.customer_id
GROUP BY Customers.name
HAVING total_orders > 1
ORDER BY total_orders DESC;
'''

result = pd.read_sql_query(query, connection)
result


,name,total_orders
0,Alice Smith,2


In [10]:
connection.close()


Customer Insights: You can understand customer behavior by analyzing metrics like Customer Lifetime Value (LTV), Average Order Value (AOV), and retention rates.

Product Performance: Identify which products are performing well and which might need better marketing or discounts.

Segmentation Analysis: Analyze differences in behavior between different customer segments (e.g., Premium vs. Regular).

Trends Analysis: Gain insights into monthly sales trends to understand seasonal effects and growth patterns.

Real-World Application: This project is relevant for any e-commerce business looking to optimize its customer acquisition, retention, and product strategy.